# Sentiment Analysis Pipeline: Twitter-RoBERTa-Base

**Objective**: Apply `cardiffnlp/twitter-roberta-base-sentiment-latest` to the Fed Speeches dataset using a standardized pipeline.
**Methodology**:
1. **Load Data**: Processed sentences (`data/master/fed_master_corpus.csv`).
2. **Inference**: Classify each sentence as Hawkish, Dovish, or Neutral.
3. **Index Calculation**: Compute Net Sentiment Index using the "Score-based" formula: $Index = \frac{Dovish_{score} - Hawkish_{score}}{Total_{count}}$.
4. **Aggregation**: Aggregate by Date/Meeting.


In [16]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from tqdm.auto import tqdm
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure project root is in path for utils import
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import custom utilities
try:
    from utils.utilities import calculate_net_sentiment_scores, get_sentiment_label_RoBERTa
    print("Successfully imported utilities.")
except ImportError as e:
    print(f"Import Error: {e}. Please ensure 'utils' package is available.")

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

# Config
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"

INPUT_FILE = "../data/master/fed_master_corpus.csv"
OUTPUT_FILE = "../data/result/RoBERTa_Base/Twitter_RoBERTa_inference_results.csv"
INDEX_OUTPUT_FILE = "../data/result/RoBERTa_Base/monthly_index_Twitter_RoBERTa.csv"

# Set Device
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {device} ({torch.cuda.get_device_name(0) if device==0 else 'CPU'}) ")

Successfully imported utilities.
Using device: -1 (CPU) 


In [17]:

    
df = pd.read_csv(INPUT_FILE)
print(f"Loaded {len(df)} sentences.")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
df.head()

Loaded 7471 sentences.
Date range: 2018-01-31 to 2024-12-18


,date,text,section,source,speaker,word_count,month_year
0,2018-01-31,The manager of the System Open Market Account ...,Developments in Financial Markets,Minutes,NaN,24,2018-01
1,2018-01-31,Domestic financial market conditions eased con...,Staff Review of Financial Situation,Minutes,NaN,11,2018-01
2,2018-01-31,A strengthening outlook for economic growth in...,Staff Review of Financial Situation,Minutes,NaN,23,2018-01
3,2018-01-31,"U.S. equity prices, Treasury yields, and marke...",Staff Review of Financial Situation,Minutes,NaN,31,2018-01
4,2018-01-31,"In addition, the dollar depreciated broadly am...",Staff Review of Financial Situation,Minutes,NaN,29,2018-01


In [18]:
# 2. Initialize Twitter-RoBERTa-Base Model Pipeline
print(f"Loading Twitter-RoBERTa-Base model: {MODEL_NAME}...")
try:
    nlp = pipeline("sentiment-analysis", 
                   model=MODEL_NAME, 
                   tokenizer=MODEL_NAME, 
                   device=device, 
                   truncation=True, 
                   max_length=512,
                   return_all_scores=False)
    print("Twitter-RoBERTa-Base model loaded successfully.")
    print(f"Model details: {nlp.model.config}")
except Exception as e:
    print(f"ERROR Loading Twitter-RoBERTa-Base Model: {e}")
    nlp = None

Loading Twitter-RoBERTa-Base model: cardiffnlp/twitter-roberta-base-sentiment-latest...


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Twitter-RoBERTa-Base model loaded successfully.
Model details: RobertaConfig {
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "dtype": "float32",
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



d:\Users\Administrator\anaconda3\envs\textmining\Lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [19]:
# 3. Run Twitter-RoBERTa-Base Inference
BATCH_SIZE = 16  # Smaller batch size for Twitter-RoBERTa-Base-large
sentences = df['text'].astype(str).tolist()
results = []

if nlp:
    print(f"Starting Twitter-RoBERTa-Base inference on {len(sentences)} sentences...")
    
    for i in tqdm(range(0, len(sentences), BATCH_SIZE)):
        batch = sentences[i:i + BATCH_SIZE]
        try:
            preds = nlp(batch)
            results.extend(preds)
        except Exception as e:
            print(f"Error at batch {i}: {e}")
            results.extend([{'label': 'LABEL_1', 'score': 0.0}] * len(batch))  # Neutral fallback
    
    print(f"Twitter-RoBERTa-Base inference completed. Processed {len(results)} predictions.")
    
else:
    print("Twitter-RoBERTa-Base model not loaded. Filling with Defaults (Neutral) for testing structure.")
    results = [{'label': 'LABEL_1', 'score': 0.0} for _ in sentences]

# Attach raw results
df['raw_sentiment'] = [x['label'] for x in results]
df['sentiment_score'] = [x['score'] for x in results]

print("\nraw_sentiment distribution:")
print(df['raw_sentiment'].value_counts())

Starting Twitter-RoBERTa-Base inference on 7471 sentences...


  0%|          | 0/467 [00:00<?, ?it/s]

Twitter-RoBERTa-Base inference completed. Processed 7471 predictions.

raw_sentiment distribution:
raw_sentiment
neutral     5113
positive    1300
negative    1058
Name: count, dtype: int64


In [20]:
# 4. Map RoBERTa Labels to Fed Context
# Using imported function: get_sentiment_label_RoBERTa

df['sentiment'] = df['raw_sentiment'].apply(get_sentiment_label_RoBERTa)

print("Mapped sentiment distribution:")
print(df['sentiment'].value_counts())

# Show mapping examples
print("\nLabel mapping examples:")
mapping_examples = df[['raw_sentiment', 'sentiment']].drop_duplicates().head(10)
for _, row in mapping_examples.iterrows():
    print(f"{row['raw_sentiment']} → {row['sentiment']}")

Mapped sentiment distribution:
sentiment
Neutral    5113
Dovish     1300
Hawkish    1058
Name: count, dtype: int64

Label mapping examples:
neutral → Neutral
positive → Dovish
negative → Hawkish


In [21]:
# 5. Calculate Sentiment Index
# Using score-based calculation: (Dovish_score - Hawkish_score) / count

df['sentiment'] = df['raw_sentiment'].apply(get_sentiment_label_RoBERTa)

print("Mapped sentiment distribution:")
print(df["sentiment"].value_counts())

# Show mapping examples
print("\nLabel mapping examples:")
mapping_examples = df[['raw_sentiment', 'sentiment']].drop_duplicates().head(10)
for _, row in mapping_examples.iterrows():
    print(f"{row['raw_sentiment']} -> {row['sentiment']}")

# Calculate monthly sentiment index
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.to_period('M')

# Apply score-based calculation
monthly_index = df.groupby("month", group_keys=False).apply(
    calculate_net_sentiment_scores, include_groups=False
).reset_index(name="sentiment_score")

monthly_index['month'] = monthly_index['month'].dt.to_timestamp()
monthly_index = monthly_index.sort_values('month')

print(f"Monthly index calculated for {len(monthly_index)} months")
print(f"RoBERTa sentiment range: {monthly_index['sentiment_score'].min():.3f} to {monthly_index['sentiment_score'].max():.3f}")

# Save results
df.to_csv(OUTPUT_FILE, index=False)
monthly_index.to_csv(INDEX_OUTPUT_FILE, index=False)
print(f"\nSaved detailed results to {OUTPUT_FILE}")
print(f"Saved monthly index to {INDEX_OUTPUT_FILE}")


Mapped sentiment distribution:
sentiment
Neutral    5113
Dovish     1300
Hawkish    1058
Name: count, dtype: int64

Label mapping examples:
neutral -> Neutral
positive -> Dovish
negative -> Hawkish
Monthly index calculated for 74 months
RoBERTa sentiment range: -0.640 to 0.707

Saved detailed results to ../data/result/RoBERTa_Base/Twitter_RoBERTa_inference_results.csv
Saved monthly index to ../data/result/RoBERTa_Base/monthly_index_Twitter_RoBERTa.csv
